<a href="https://colab.research.google.com/github/niladri-rkmvu/dsa-2025/blob/10.sorting/sorting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# All Sorting Algorithms

In [6]:
import random

class Sorter:
    def __init__(self, A):
        # Initialize Sorter with a copy of the input array
        self.A = list(A)

    def print_array(self):
        # Print current array elements space-separated
        print(*self.A)

    def swap(self, i, j):
        # Swap elements at indices i and j
        self.A[i], self.A[j] = self.A[j], self.A[i]

    def reset(self, new_A):
        # Reset array to a new copy (for re-sorting)
        self.A = list(new_A)

    # ---------------- Bubble Sort ----------------
    def bubble_sort(self):
        # Repeatedly swap adjacent elements if out of order
        # Optimized with a flag to stop early if already sorted
        n = len(self.A)
        for i in range(n - 1):
            flag = False
            for j in range(n - i - 1):
                if self.A[j] > self.A[j + 1]:
                    self.swap(j, j + 1)
                    flag = True
            if not flag:  # No swaps → array sorted
                break

    # ---------------- Insertion Sort ----------------
    def insertion_sort(self):
        # Build sorted subarray by inserting each element in correct position
        n = len(self.A)
        for i in range(1, n):
            key = self.A[i]
            j = i - 1
            # Shift larger elements right until correct position found
            while j >= 0 and self.A[j] > key:
                self.A[j + 1] = self.A[j]
                j -= 1
            self.A[j + 1] = key

    # ---------------- Selection Sort ----------------
    def selection_sort(self):
        # Select minimum element from unsorted part and place at beginning
        n = len(self.A)
        for i in range(n - 1):
            min_idx = i
            for j in range(i + 1, n):
                if self.A[j] < self.A[min_idx]:
                    min_idx = j
            self.swap(i, min_idx)

    # ---------------- Quick Sort ----------------
    def partition(self, l, h):
        # Partition array around pivot (first element)
        pivot = self.A[l]
        i, j = l, h
        while i < j:
            while i <= h and self.A[i] <= pivot:
                i += 1
            while j >= l and self.A[j] > pivot:
                j -= 1
            if i < j:
                self.swap(i, j)
        # Place pivot in correct position
        self.swap(l, j)
        return j

    def quick_sort(self, l=0, h=None):
        # Recursive quicksort using partition
        if h is None:
            h = len(self.A) - 1
        if l < h:
            j = self.partition(l, h)
            self.quick_sort(l, j - 1)   # Sort left partition
            self.quick_sort(j + 1, h)   # Sort right partition

    # ---------------- Merge Sort ----------------
    def merge(self, l, mid, h):
        # Merge two sorted halves into one sorted segment
        i, j = l, mid + 1
        temp = []

        # Compare elements from both halves
        while i <= mid and j <= h:
            if self.A[i] < self.A[j]:
                temp.append(self.A[i]); i += 1
            else:
                temp.append(self.A[j]); j += 1

        # Copy remaining elements
        while i <= mid:
            temp.append(self.A[i]); i += 1
        while j <= h:
            temp.append(self.A[j]); j += 1

        # Write merged result back to array
        for k in range(len(temp)):
            self.A[l + k] = temp[k]

    def merge_sort_iter(self):
        # Iterative (bottom-up) merge sort
        n = len(self.A)
        p = 1
        while p < n:
            for i in range(0, n - p, 2 * p):
                l = i
                mid = l + p - 1
                h = min(l + 2 * p - 1, n - 1)
                self.merge(l, mid, h)
            p *= 2

    def merge_sort_recursive(self, l=0, h=None):
        # Recursive (top-down) merge sort
        if h is None:
            h = len(self.A) - 1
        if l < h:
            mid = (l + h) // 2
            self.merge_sort_recursive(l, mid)      # Sort left half
            self.merge_sort_recursive(mid + 1, h)  # Sort right half
            self.merge(l, mid, h)                  # Merge halves

    # ---------------- Count Sort ----------------
    def count_sort(self):
        # Count sort works for non-negative integers
        max_val = max(self.A)

        # Step 1: Create count array
        count = [0] * (max_val + 1)

        # Step 2: Store frequencies
        for num in self.A:
            count[num] += 1

        # Step 3: Write back sorted values
        i = 0
        for j in range(max_val + 1):
            while count[j] > 0:
                self.A[i] = j
                i += 1
                count[j] -= 1

    # ---------------- Unified Bucket Sort ----------------
    def bucket_sort(self, num_bins=None):
        n = len(self.A)
        if n == 0:
            return

        # Step 1: Decide number of bins
        if num_bins is None:
            num_bins = n  # default: one bin per element

        # Step 2: Find min and max values
        min_val, max_val = min(self.A), max(self.A)

        # Step 3: Compute bin size (range per bin)
        if max_val > min_val:
            bin_size = (max_val - min_val) / num_bins
        else:
            bin_size = 1

        # Step 4: Create empty bins
        bins = [[] for _ in range(num_bins)]

        # Step 5: Distribute elements into bins
        for num in self.A:
            index = int((num - min_val) / bin_size)
            if index >= num_bins:  # edge case for max value
                index = num_bins - 1
            bins[index].append(num)

        # Step 6: Sort each bin using the existing insertion_sort
        for b in bins:
            if b:  # only sort non-empty bins
                temp_sorter = Sorter(b)   # reuse Sorter class
                temp_sorter.insertion_sort()
                b[:] = temp_sorter.A      # copy back sorted bucket

        # Step 7: Concatenate bins back into array
        i = 0
        for b in bins:
            for num in b:
                self.A[i] = num
                i += 1

    # ---------------- Radix Sort ----------------
    def _bin_sort_by_digit(self, exp):
        n = len(self.A)

        # Create 10 bins (buckets) for digits 0 through 9
        bins = [[] for _ in range(10)]

        # Step 1: Distribute (Scatter) elements into bins
        for num in self.A:
            # Formula to isolate the digit: (number // exponent) % 10
            digit = (num // exp) % 10
            bins[digit].append(num)

        # Step 2: Collect (Gather) elements from bins back into A
        # This step is inherently stable because we process the original array
        # elements sequentially as they were placed, preserving their order within each bin.
        i = 0
        for b in bins:
            # The inner loop effectively concatenates the bins
            for num in b:
                self.A[i] = num
                i += 1

    def _counting_sort_by_digit(self, exp):
        n = len(self.A)
        output = [0] * n   # temporary output array
        count = [0] * 10   # count array for digits 0-9

        # Step 1: Count occurrences of each digit
        for i in range(n):
            digit = (self.A[i] // exp) % 10
            count[digit] += 1

        # Step 2: Convert count to cumulative count
        for i in range(1, 10):
            count[i] += count[i - 1]

        # Step 3: Build output array (stable placement)
        i = n - 1
        while i >= 0:
            digit = (self.A[i] // exp) % 10
            output[count[digit] - 1] = self.A[i]
            count[digit] -= 1
            i -= 1

        # Step 4: Copy output back to A
        for i in range(n):
            self.A[i] = output[i]

    def radix_sort(self):
        # Step 1: Find maximum number to know number of digits
        max_val = max(self.A)

        # Step 2: Apply bin sort for each digit / if you want apply count_sort too
        # we are discussing bin sort in class.
        exp = 1
        while max_val // exp > 0:
            self._bin_sort_by_digit(exp)
            # self._counting_sort_by_digit(exp)
            exp *= 10

    # Student exercise to trace the code
    def shell_sort(self):
        n = len(self.A)
        gap = n // 2
        while gap > 0:
            # For each starting position within the gap
            for start in range(gap):
                # Step 1: Extract subsequence with step = gap
                subseq = []
                for i in range(start, n, gap):
                    subseq.append(self.A[i])

                # Step 2: Sort subsequence using existing insertion_sort
                temp_sorter = Sorter(subseq)   # reuse your class
                temp_sorter.insertion_sort()
                subseq_sorted = temp_sorter.A

                # Step 3: Put sorted subsequence back into original array
                for idx, val in enumerate(subseq_sorted):
                    self.A[start + idx * gap] = val
            gap //= 2

if __name__ == "__main__":
    A = [11, 13, 7, 12, 16, 9, 24, 5, 10, 3]
    random.shuffle(A)
    sorter = Sorter(A)

    print("Unsorted Array:")
    sorter.print_array()

    # Bubble Sort
    sorter.reset(A)
    sorter.bubble_sort()
    print("Bubble Sorted Array:")
    sorter.print_array()

    # Insertion Sort
    sorter.reset(A)
    sorter.insertion_sort()
    print("Insertion Sorted Array:")
    sorter.print_array()

    # Selection Sort
    sorter.reset(A)
    sorter.selection_sort()
    print("Selection Sorted Array:")
    sorter.print_array()

    # Quick Sort
    sorter.reset(A)
    sorter.quick_sort()
    print("Quick Sorted Array:")
    sorter.print_array()

    # Merge Sort (Iterative)
    sorter.reset(A)
    sorter.merge_sort_iter()
    print("Merge Sorted Array (Iterative):")
    sorter.print_array()

    # Merge Sort (Recursive)
    sorter.reset(A)
    sorter.merge_sort_recursive()
    print("Merge Sorted Array (Recursive):")
    sorter.print_array()

    # Count Sort
    sorter.reset(A)
    sorter.count_sort()
    print("Count Sorted Array:")
    sorter.print_array()

    # Bucket Sort
    sorter.reset(A)
    sorter.bucket_sort(5)
    print("Bucket Sorted Array:")
    sorter.print_array()

    # Radix Sort
    sorter.reset(A)
    sorter.radix_sort()
    print("Radix Sorted Array:")
    sorter.print_array()

    # Shell Sort
    sorter.reset(A)
    sorter.shell_sort()
    print("Shell Sorted Array:")
    sorter.print_array()

Unsorted Array:
9 7 3 24 16 10 11 5 12 13
Bubble Sorted Array:
3 5 7 9 10 11 12 13 16 24
Insertion Sorted Array:
3 5 7 9 10 11 12 13 16 24
Selection Sorted Array:
3 5 7 9 10 11 12 13 16 24
Quick Sorted Array:
3 5 7 9 10 11 12 13 16 24
Merge Sorted Array (Iterative):
3 5 7 9 10 11 12 13 16 24
Merge Sorted Array (Recursive):
3 5 7 9 10 11 12 13 16 24
Count Sorted Array:
3 5 7 9 10 11 12 13 16 24
Bucket Sorted Array:
3 5 7 9 10 11 12 13 16 24
Radix Sorted Array:
3 5 7 9 10 11 12 13 16 24
Shell Sorted Array:
3 5 7 9 10 11 12 13 16 24
